In [271]:
#BP神经网络
%matplotlib qt5
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import Callback
from sklearn.preprocessing import scale
import matplotlib.pyplot as plt  
from mpl_toolkits.mplot3d import Axes3D  

In [272]:
#训练数据
X = []
Y = []
Z = []
#path = '/Users/liuyingjie/Downloads/data/multi/1/'
path = '/Users/liuyingjie/Downloads/data/two/5/'
filenames = os.listdir(path)
indexs = []
for filename in filenames:
    if filename.startswith("."):
        continue
    indexs.append(int(filename))
indexs.sort()
#print(indexs)
for index in indexs:
    file_path = path + str(index)
    with open(file_path,'r',encoding='utf8') as f:
        lines=f.readlines()
    for num in range(102, 122):
        #print(lines[num])
        temps = lines[num].split()
        X.append(float(temps[2])/100)
        Y.append(float(temps[3])/100)
    for line in lines:
        temps = line.split()
        size = len(temps)
        if(size == 11 and temps[0]=='10000000'):
            Z.append(int(float(temps[1])*37000000000))
            Z.append(int(float(temps[6])*37000000000))
        if(size == 16 and temps[0] == '10000000'):
            Z.append(int(float(temps[1])*37000000000))
            Z.append(int(float(temps[6])*37000000000))
            Z.append(int(float(temps[11])*37000000000))
            
       

In [273]:
#预测数据
Point1=[90,110]#预测场左上角坐标
Point2=[110,90]#预测场右下角坐标
X_interval=0.3
Y_interval=-0.3
X_p = []
Y_p = []
X_start=Point1[0]-X_interval
Y_start=Point1[1]-Y_interval
while abs(Point2[0] -X_start)>=abs(X_interval)or abs(Point2[1]-Y_start)>=abs(Y_interval):
    X_start=Point1[0]-X_interval
    Y_start+=Y_interval
    while abs(Point2[0] -X_start)>=abs(X_interval):
        X_start+=X_interval
        X_p.append(X_start)
        Y_p.append(Y_start)
#print(X_p)
#print(Y_p)
#print(len(X_p))
#print(len(Y_p))

In [274]:
X_pred = scale(np.array(X_p))
Y_pred = scale(np.array(Y_p))
X_pred_ = np.array([X_pred,Y_pred])
X = scale(np.array(X))
Y = scale(np.array(Y))
X_ = np.array([X,Y])
Y_ = scale(np.array(Z))

In [290]:
from keras.layers import Dense, Activation,Dropout
from keras.layers import LeakyReLU,PReLU,ELU
from keras import regularizers
model = Sequential()
model.add(Dense(units = 128, input_dim = 2))
model.add(ELU(alpha=0.01))
model.add(Dropout(0.3))
model.add(Dense(units = 128))
model.add(ELU(alpha=0.01))
model.add(Dropout(0.3))
model.add(Dense(units = 1))
model.add(ELU(alpha=0.01))
#model.add(LeakyReLU(alpha=0.1))
#model.add(Activation("tanh"))
model.compile(loss='mean_squared_error', 
              optimizer='Adam', #Adam
              metrics=['mae'])
model.fit(X_.T, Y_.reshape(-1,1), epochs = 100, batch_size =40, shuffle=True, verbose = 0)
Y_pred = model.predict(X_pred_.T)

ans=[]
predict=[]
xLength=int(abs(Point2[0]-Point1[0])/abs(X_interval))
yLength=int(abs(Point2[1]-Point1[1])/abs(Y_interval))
i=0
while i <len(X_p):
    j=0
    res=[]
    while j<=xLength:
        r=[X_p[i],Y_p[i],Y_pred[i]]
        res.append(r)
        i+=1
        j+=1
    ans.append(res)
#print(ans) 
res=[]
i=1
while i<yLength:
    j=1
    while j<xLength:
        if ans[i][j][2]>ans[i-1][j][2] and ans[i][j][2]>ans[i+1][j][2] and ans[i][j][2]>ans[i][j-1][2] \
           and ans[i][j][2]>ans[i][j+1][2] and ans[i][j][2]>ans[i-1][j-1][2] and ans[i][j][2]>ans[i-1][j+1][2] \
            and ans[i][j][2]>ans[i+1][j-1][2] and ans[i][j][2]>ans[i+1][j+1][2]:
            res.append(ans[i][j])
        j+=1
    i+=1
res.sort(key=lambda k:k[2],reverse = True)
print(res[:4])

[[94.49999999999996, 97.40000000000012, array([2.7892683], dtype=float32)], [101.99999999999989, 105.80000000000004, array([2.4441369], dtype=float32)]]


In [291]:
ax = plt.figure().add_subplot(111, projection = '3d')  
ax.scatter(X_pred_[0], X_pred_[1], Y_pred, c = 'b', marker = 'o') #点为红色三角形  
ax.set_xlabel('X Label')  
ax.set_ylabel('Y Label')  
ax.set_zlabel('Z Label')  
plt.show()

In [102]:
ax = plt.figure().add_subplot(111, projection = '3d')  
ax.scatter(X_[0], X_[1], Y_, c = 'b', marker = 'o') #点为红色三角形  
ax.set_xlabel('X Label')  
ax.set_ylabel('Y Label')  
ax.set_zlabel('Z Label')  
plt.show()

In [8]:
Y_ = scale(np.array(Z))
# X = [98+i/12 for i in range(48)]
# Y = [98+i/12 for i in range(48)]
# X, Y = np.meshgrid(X,Y)
# X = X.reshape(2304,)
# Y = Y.reshape(2304,)
# X = np.array(X)
# Y = np.array(Y)
# Y_ = [1/((X[i]-99.8)**2+(Y[i]-99.8)**2+.5) for i in range(2304)]
# Y_ = np.array(Y_)
# X = scale(X)
# Y = scale(Y)
# X_ = np.array([X,Y])
#print(Y_)
ax = plt.figure().add_subplot(111, projection = '3d')  
ax.scatter(X_[0], X_[1], Y_, c = 'b', marker = '.') #点为红色三角形   
ax.set_xlabel('X Label')  
ax.set_ylabel('Y Label')  
ax.set_zlabel('Z Label')  
plt.show()
X_.shape

(2, 80)

In [225]:
from keras.layers import Dense, Activation
from keras.layers import LeakyReLU,PReLU,ELU
model = Sequential()
model.add(Dense(units = 128, input_dim = 2))
model.add(Activation("relu"))
model.add(Dense(units = 64))
model.add(Activation("relu"))
model.add(Dense(units = 64))
model.add(Activation("relu"))
model.add(Dense(units = 1))
model.add(Activation("linear"))
model.compile(loss='mean_squared_error', 
              optimizer='Adam', #Adam
              metrics=['mae'])
model.fit(X_.T, Y_.reshape((-1,1)), epochs = 100, batch_size =20, shuffle=True, verbose = 0)
Y_pred = model.predict(X_pred_.T)
max,ans=-123456,0
for i in range(len(Y_pred)):
    if Y_pred[i]>max:
        ans=i
        max=Y_pred[i]
print(X_p[ans])
print(Y_p[ans])

94.79999999999995
97.40000000000012


In [226]:
ans=[]
predict=[]
xLength=int(abs(Point2[0]-Point1[0])/abs(X_interval))
yLength=int(abs(Point2[1]-Point1[1])/abs(Y_interval))
i=0
while i <len(X_p):
    j=0
    res=[]
    while j<=xLength:
        r=[X_p[i],Y_p[i],Y_pred[i][0]]
        res.append(r)
        i+=1
        j+=1
    ans.append(res)
#print(ans) 
res=[]
i=1
while i<yLength:
    j=1
    while j<xLength:
        if ans[i][j][2]>ans[i-1][j][2] and ans[i][j][2]>ans[i+1][j][2] and ans[i][j][2]>ans[i][j-1][2] \
           and ans[i][j][2]>ans[i][j+1][2] and ans[i][j][2]>ans[i-1][j-1][2] and ans[i][j][2]>ans[i-1][j+1][2] \
            and ans[i][j][2]>ans[i+1][j-1][2] and ans[i][j][2]>ans[i+1][j+1][2]:
            res.append(ans[i][j])
        j+=1
    i+=1
res.sort(key=lambda k:k[2],reverse = True)
print(res[:4])

[[94.79999999999995, 97.40000000000012, 5.1497083], [101.39999999999989, 105.20000000000005, 4.473132]]


In [11]:
ans=[]
predict=[]
for i in range(len(X_p)):
    res=[X_p[i],Y_p[i],Y_pred[i][0]]
    ans.append(res)
ans.sort(key=lambda k:k[2],reverse = True)
predict.append(ans[0])
#print(predict)
"""
for help in ans:
    if abs(help[0]-predict[0][0])<6 and abs(help[1]-predict[0][1])<6:
        help[2]=0
ans.sort(key=lambda k:k[2],reverse = True)
"""
print(ans)

[[90.0, 96.50000000000013, 3.4165525], [90.0, 96.80000000000013, 3.3750377], [90.0, 96.20000000000013, 3.373646], [90.3, 96.50000000000013, 3.321966], [90.3, 96.80000000000013, 3.3129354], [90.0, 95.90000000000013, 3.2892027], [90.0, 97.10000000000012, 3.2608972], [90.3, 96.20000000000013, 3.2597418], [90.6, 96.80000000000013, 3.2442255], [90.3, 97.10000000000012, 3.221349], [90.6, 96.50000000000013, 3.2175784], [90.6, 97.10000000000012, 3.179881], [90.3, 95.90000000000013, 3.1615224], [90.89999999999999, 96.80000000000013, 3.160482], [90.0, 95.60000000000014, 3.1564474], [90.6, 96.20000000000013, 3.1379976], [90.89999999999999, 97.10000000000012, 3.127911], [90.0, 97.40000000000012, 3.1124501], [90.89999999999999, 96.50000000000013, 3.107162], [90.3, 97.40000000000012, 3.0824542], [91.19999999999999, 97.10000000000012, 3.065239], [91.19999999999999, 96.80000000000013, 3.0598545], [90.6, 97.40000000000012, 3.0483675], [90.6, 95.90000000000013, 3.0158682], [90.89999999999999, 97.4000000

In [8]:
ax = plt.figure().add_subplot(111, projection = '3d')  
ax.scatter(X_pred_[0], X_pred_[1], Y_pred, c = 'b', marker = 'o') #点为红色三角形  
ax.set_xlabel('X Label')  
ax.set_ylabel('Y Label')  
ax.set_zlabel('Z Label')  
plt.show()